In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:02<00:00,  1.44it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [45]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_train = df_train.join(hist_max_date).join(new_min_date)
df_test = df_test.join(hist_max_date).join(new_min_date)
df_train['hist_purchase_month_max'] = pd.to_datetime(df_train['hist_purchase_month_max'])
df_train['first_active_month'] = pd.to_datetime(df_train['first_active_month'])
df_test['hist_purchase_month_max'] = pd.to_datetime(df_test['hist_purchase_month_max'])
df_test['first_active_month'] = pd.to_datetime(df_test['first_active_month'])
df_train['hist_regist_term']= (df_train['hist_purchase_month_max'].map(lambda x: x.year) - df_train['first_active_month'].map(lambda x: x.year)) * 12 + (df_train['hist_purchase_month_max'].map(lambda x: x.month) - df_train['first_active_month'].map(lambda x: x.month))
df_test['hist_regist_term']= (df_test['hist_purchase_month_max'].map(lambda x: x.year) - df_test['first_active_month'].map(lambda x: x.year)) * 12 + (df_test['hist_purchase_month_max'].map(lambda x: x.month) - df_test['first_active_month'].map(lambda x: x.month))
train_test = pd.concat([df_train, df_test], axis=0)
train_test.head()

In [8]:
df = df_hist
auth1 = df[df.authorized_flag==1]

cat1_0 = False
# catt1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()
    auth1_cat1_lag0  = auth1_cat1[auth1_cat1['month_lag']==0]
    auth1_cat0_lag0  = auth1_cat0[auth1_cat0['month_lag']==0]
    auth1_cat1_lag1  = auth1_cat1[auth1_cat1['month_lag']==-1]
    auth1_cat0_lag1  = auth1_cat0[auth1_cat0['month_lag']==-1]
    auth1_cat1_lag2  = auth1_cat1[auth1_cat1['month_lag']==-2]
    auth1_cat0_lag2  = auth1_cat0[auth1_cat0['month_lag']==-2]
    auth1_cat1_lag02 = auth1_cat1[auth1_cat1['month_lag']>=-2]
    auth1_cat0_lag02 = auth1_cat0[auth1_cat0['month_lag']>=-2]
    auth1_cat1_lag05 = auth1_cat1[auth1_cat1['month_lag']>=-5]
    auth1_cat0_lag05 = auth1_cat0[auth1_cat0['month_lag']>=-5]
    
    new_cat1_lag1  = new_cat1[new_cat1['month_lag']==1]
    new_cat0_lag1  = new_cat0[new_cat0['month_lag']==1]
    new_cat1_lag2  = new_cat1[new_cat1['month_lag']==2]
    new_cat0_lag2  = new_cat0[new_cat0['month_lag']==2]

In [9]:
# 未来の残高を計算する
# df_trans = pd.concat([df, auth1], axis=0)
use_cols = ['card_id', 'purchase_amount', 'installments', 'category_1', 'category_2', 'merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'first_active_month', 'yyyymm', 'yyyymmdd']
df_trans = df_trans[use_cols]
df_trans = df_trans[df_trans['installments']>=2]
df_trans['installments'].value_counts()

2.0     1255584
3.0     1002278
4.0      326718
6.0      236053
5.0      210028
10.0     202246
12.0      91059
8.0       34651
7.0       18466
9.0        9603
11.0       1378
Name: installments, dtype: int64

In [39]:
ym = 'yyyymm'
df_trans['yyyymm'] = pd.to_datetime(df_trans['yyyymm'])

ins_list = list(df_trans['installments'].value_counts().index)
df_list = []

for ins in ins_list:

    tmp = df_trans[df_trans['installments']==ins]
    
    # installments==2とした当月
    id_ym = tmp[[key, 'yyyymm', 'purchase_amount']].groupby([key, 'yyyymm'])['purchase_amount'].sum().reset_index()
    # 分割あたりのamount
    id_ym['purchase_amount'] = id_ym['purchase_amount'] / ins
    
    # 未来の月に対するamountをもったデータ
    df_ins_ym = id_ym[[key, 'yyyymm', 'purchase_amount']].copy()
    
    for i in range(int(ins-1)):
        df_ins_ym['yyyymm'] = df_ins_ym['yyyymm'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
        df_ins_ym['yyyymm'] = df_ins_ym['yyyymm'].map(lambda x: str(x)[:7])
        df_ins_ym['yyyymm'] = pd.to_datetime(df_ins_ym['yyyymm'])
        df_list.append(df_ins_ym.copy())
        
future_amt = pd.concat(df_list, axis=0, ignore_index=True)
print(future_amt.shape)
display(future_amt.head())

(3647284, 3)


,card_id,yyyymm,purchase_amount
0,C_ID_00007093c1,2017-03-01,0.784180
1,C_ID_00007093c1,2017-07-01,0.447754
2,C_ID_00007093c1,2017-08-01,0.226929
3,C_ID_00007093c1,2017-09-01,0.097656
4,C_ID_00007093c1,2017-11-01,0.105164


In [59]:
#========================================================================
# train_testにつけたhist_max_month, new_min_monthをつける
join_cols = ['hist_purchase_month_max', 'new_purchase_month_min', 'first_active_month']

print(future_amt.shape)
df_ins = future_amt.set_index(key).join(train_test[join_cols])
print(df_ins.shape)
#========================================================================

#========================================================================
# new_month_minとの相対月をつけ、それを元にデータを絞り計算する
df_ins['yyyymm'] = pd.to_datetime(df_ins['yyyymm'])
df_ins['new_purchase_month_min'] = pd.to_datetime(df_ins['new_purchase_month_min'])
df_ins['new_month_lag']= (df_ins['new_purchase_month_min'].map(lambda x: x.year) - df_ins['yyyymm'].map(lambda x: x.year)) * 12 + (df_ins['new_purchase_month_min'].map(lambda x: x.month) - df_ins['yyyymm'].map(lambda x: x.month))
df_ins['new_month_lag'] *= -1
df_ins['hist_purchase_month_max'] = pd.to_datetime(df_ins['hist_purchase_month_max'])
df_ins['hist_month_lag']= (df_ins['hist_purchase_month_max'].map(lambda x: x.year) - df_ins['yyyymm'].map(lambda x: x.year)) * 12 + (df_ins['hist_purchase_month_max'].map(lambda x: x.month) - df_ins['yyyymm'].map(lambda x: x.month))
df_ins['hist_month_lag'] *= -1

# new_minとhist_maxにズレがあるとcard_idによって条件が異なってしまうので、newのデータがないcard_id以外はnewのlagにする
df_ins['new_month_lag'].fillna(-999, inplace=True)
df_ins['org_month_lag'] = df_ins[['new_month_lag', 'hist_month_lag']].min(axis=1)
df_ins['new_month_lag'] = df_ins['new_month_lag'].replace(-999, np.nan)
#========================================================================

In [66]:
# 期間別のデータを作る
df_ins_new = df_ins[df_ins['org_month_lag']>=0]
df_ins_past3 = df_ins[df_ins['org_month_lag']<0]
df_ins_past3 = df_ins_past3[df_ins_past3['org_month_lag']>=-3]
df_ins_past6 = df_ins[df_ins['org_month_lag']<0]
df_ins_past6 = df_ins_past6[df_ins_past6['org_month_lag']>=-6]

In [73]:
prefix = '215_ins'
df_list = [df_ins, df_ins_new, df_ins_past3, df_ins_past6]
fname_list = ['rem_ins', 'rem_ins_new', 'rem_ins_past3', 'rem_ins_past6']
agg_cols = [key, 'org_month_lag', 'cnt', 'purchase_amount']

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(tqdm(df_list), fname_list):
    
    # Base
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # FE
    df.reset_index(inplace=True)
    df['cnt'] = 1
    df = df[agg_cols]
    # 月あたりの分割支払決済数
    tmp = df.groupby([key, 'org_month_lag']).sum().reset_index()
    aggs = {}
    aggs['cnt'] = ['sum', 'mean', 'max', 'min', 'std']
    aggs['purchase_amount'] = ['sum', 'mean', 'max', 'min', 'std']
    # 分割の最終月、初月、平均
    aggs['org_month_lag'] = ['max', 'min', 'mean']
    df_agg = tmp.groupby(key)['cnt'].agg(aggs)
    
    new_cols = get_new_columns(fname, aggs)
    df_agg.columns = new_cols
    
    train_test = train_test.join(df_agg)
    
    for col in train_test.columns:
        if not(col.count('rem_ins')):continue
        feature = train_test[col].astype('float32').values
        if np.std(feature)==0:
            continue
        
    #     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)

100%|██████████| 4/4 [00:12<00:00,  3.17s/it]


In [78]:
import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
fname_list = ['rem_ins', 'rem_ins_new', 'rem_ins_past3', 'rem_ins_past6']

combi_list = [ 
    ['rem_ins_new', 'rem_ins_past3']
    ,['rem_ins_new', 'rem_ins_past6']
    ,['rem_ins_past3', 'rem_ins_past6']
]

comp_cols = [
     'cnt_sum'
    ,'cnt_max'
    ,'purchase_amount_sum'
    ,'purchase_amount_max'
    ,'org_month_lag_min'
    ,'org_month_lag_max'
]

tmp_feature_list = glob.glob('../features/3_third_valid/215_*.gz')  + glob.glob('../features/4_winner/215_*.gz') + glob.glob('../features/1_first_valid/215_*.gz') + glob.glob('../features/2_second_valid/215_*.gz')
                   
feature_list = []
for f in tmp_feature_list:
    if not(f.count('215_ins_')):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
            print(fm1, fm2, col)
            continue
print(train_test.shape)
train_test.head()

100%|██████████| 3/3 [00:00<00:00, 65.46it/s]

(325540, 61)


,card_id,215_ins_rem_ins_past3_org_month_lag_max@,215_ins_rem_ins_org_month_lag_max@,215_ins_rem_ins_org_month_lag_min@,215_ins_rem_ins_past3_purchase_amount_sum@,215_ins_rem_ins_purchase_amount_max@,215_ins_rem_ins_past3_org_month_lag_min@,215_ins_rem_ins_new_purchase_amount_max@,215_ins_rem_ins_new_cnt_sum@,215_ins_rem_ins_new_purchase_amount_sum@,215_ins_rem_ins_new_org_month_lag_max@,215_ins_rem_ins_past6_purchase_amount_sum@,215_ins_rem_ins_past3_purchase_amount_max@,215_ins_rem_ins_cnt_max@,215_ins_rem_ins_past6_cnt_sum@,215_ins_rem_ins_purchase_amount_sum@,215_ins_rem_ins_new_org_month_lag_min@,215_ins_rem_ins_cnt_sum@,215_ins_rem_ins_past3_cnt_max@,215_ins_rem_ins_past6_purchase_amount_max@,215_ins_rem_ins_past6_cnt_max@,215_ins_rem_ins_past6_org_month_lag_max@,215_ins_rem_ins_new_cnt_max@,215_ins_rem_ins_past6_org_month_lag_min@,215_ins_rem_ins_past3_cnt_sum@,ratio_rem_ins_new_rem_ins_past3_cnt_sum,diff_rem_ins_new_rem_ins_past3_cnt_sum,ratio_rem_ins_new_rem_ins_past3_cnt_max,diff_rem_ins_new_rem_ins_past3_cnt_max,ratio_rem_ins_new_rem_ins_past3_purchase_amount_sum,diff_rem_ins_new_rem_ins_past3_purchase_amount_sum,ratio_rem_ins_new_rem_ins_past3_purchase_amount_max,diff_rem_ins_new_rem_ins_past3_purchase_amount_max,ratio_rem_ins_new_rem_ins_past3_org_month_lag_min,diff_rem_ins_new_rem_ins_past3_org_month_lag_min,ratio_rem_ins_new_rem_ins_past3_org_month_lag_max,diff_rem_ins_new_rem_ins_past3_org_month_lag_max,ratio_rem_ins_new_rem_ins_past6_cnt_sum,diff_rem_ins_new_rem_ins_past6_cnt_sum,ratio_rem_ins_new_rem_ins_past6_cnt_max,diff_rem_ins_new_rem_ins_past6_cnt_max,ratio_rem_ins_new_rem_ins_past6_purchase_amount_sum,diff_rem_ins_new_rem_ins_past6_purchase_amount_sum,ratio_rem_ins_new_rem_ins_past6_purchase_amount_max,diff_rem_ins_new_rem_ins_past6_purchase_amount_max,ratio_rem_ins_new_rem_ins_past6_org_month_lag_min,diff_rem_ins_new_rem_ins_past6_org_month_lag_min,ratio_rem_ins_new_rem_ins_past6_org_month_lag_max,diff_rem_ins_new_rem_ins_past6_org_month_lag_max,ratio_rem_ins_past3_rem_ins_past6_cnt_sum,diff_rem_ins_past3_rem_ins_past6_cnt_sum,ratio_rem_ins_past3_rem_ins_past6_cnt_max,diff_rem_ins_past3_rem_ins_past6_cnt_max,ratio_rem_ins_past3_rem_ins_past6_purchase_amount_sum,diff_rem_ins_past3_rem_ins_past6_purchase_amount_sum,ratio_rem_ins_past3_rem_ins_past6_purchase_amount_max,diff_rem_ins_past3_rem_ins_past6_purchase_amount_max,ratio_rem_ins_past3_rem_ins_past6_org_month_lag_min,diff_rem_ins_past3_rem_ins_past6_org_month_lag_min,ratio_rem_ins_past3_rem_ins_past6_org_month_lag_max,diff_rem_ins_past3_rem_ins_past6_org_month_lag_max
0,C_ID_92a2005557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,12.0,13.0,1.0,12.0,7.411765,8.0,3.0,15.0,7.0,7.0,13.0,9.666667,13.0,65.0,13.0,1.0,126.0,12.0,10.833333,13.0,13.0,7.0,8.0,29.0,0.517241,-14.0,0.583333,-5.0,0.583333,-5.0,0.310345,-6.666667,0.125,-7.0,0.583333,-5.0,0.230769,-50.0,0.538462,-6.0,0.538462,-6.0,0.276923,-7.833333,0.125,-7.0,0.538462,-6.0,0.446154,-36.0,0.923077,-1.0,0.923077,-1.0,0.892308,-1.166666,1.0,0.0,0.923077,-1.0
2,C_ID_d639edf6cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,3.0,3.0,1.0,3.0,1.600000,1.0,1.5,3.0,2.0,2.0,3.0,2.000000,3.0,5.0,3.0,1.0,8.0,3.0,1.666667,3.0,3.0,2.0,1.0,4.0,0.750000,-1.0,0.666667,-1.0,0.666667,-1.0,0.750000,-0.500000,1.000,0.0,0.666667,-1.0,0.600000,-2.0,0.666667,-1.0,0.666667,-1.0,0.900000,-0.166667,1.000,0.0,0.666667,-1.0,0.800000,-1.0,1.000000,0.0,1.000000,0.0,1.200000,0.333333,1.0,0.0,1.000000,0.0
4,C_ID_cdbd2c0db2,2.0,2.0,1.0,2.0,1.076923,1.0,1.0,11.0,1.0,1.0,2.0,1.500000,2.0,3.0,2.0,1.0,14.0,2.0,1.500000,2.

In [79]:
for col in train_test.columns:
    if not(col.count('ratio_')) and not(col.count('diff_')):continue
    feature = train_test[col].astype('float32').values
    if np.std(feature)==0:
        continue
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)